In [341]:
G = CartanType('A2')
G.dynkin_diagram()

O---O
1   2   
A2

In [342]:
L = CartanType('A1')
L.dynkin_diagram()

O
1   
A1

In [343]:
crossed_out_nodes = [1]
crossed_out_nodes.sort()
crossed_out_nodes

[1]

In [344]:
G.marked_nodes(crossed_out_nodes).dynkin_diagram()

X---O
1   2   
A2 with node 1 marked

Correction function which converts index of L to that of G.

In [345]:
def correct_index(index: int) -> int:
    for i in range(len(crossed_out_nodes)):
        if index + i < crossed_out_nodes[i]:
            return index + i
    return index + len(crossed_out_nodes)

In [346]:
[ correct_index(k) for k in L.index_set() ]

[2]

In [347]:
R_G = WeylCharacterRing(G)
R_L = WeylCharacterRing(L)

In [348]:
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [349]:
R = PolynomialRing(ZZ, 'x', G.root_system().ambient_space().dimension())
R

Multivariate Polynomial Ring in x0, x1, x2 over Integer Ring

In [350]:
x = R.gens()
x

(x0, x1, x2)

The below variable `weight` means a list or tuple of coefficients, so `weight` stands for an actual weight
`sum( weight[i] * fundamental_weight[i] for i)`

In [351]:
def parabolic_weight_muliplicities(weight) -> dict:
    fws_L = [fw for fw in R_L.fundamental_weights()] # conversin from 1-index to 0-index
    weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(crossed_out_nodes)] 
    weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(L.rank()))

    fws_G = [fw for fw in R_G.fundamental_weights()]; fws_G
    weight_for_G = sum( weight[i] * fws_G[i] for i in range(G.rank()))

    mul_set = root_difference_multiplicities(R_L, weight_for_L)

    result = {}
    for k, v in mul_set.items():
        w = weight_for_G + sum(k[i - 1] * R_G.simple_roots()[correct_index(i)] for i in (1..L.rank()))
        result[w] = v

    return result

In [352]:
parabolic_weight_muliplicities((1, 1))

{(2, 1, 0): 1, (2, 0, 1): 1}

!!A-type only!!

In [353]:
def class_from_weight(character_ring, weight):
    A = matrix([ vector(sr) for sr in character_ring.fundamental_weights() ]).transpose()
    v = A.solve_right(vector(weight) - vector([weight[-1]]*len(weight)))

    return sum(v[i] * x[i] for i in range(character_ring.rank()))

In [354]:
class_from_weight(R_G, (2, 0, 1))

2*x0 - x1

In [355]:
class_from_weight(R_G, (2, 1, 0))

x0 + x1

In [356]:
parabolic_weight_muliplicities((1, 1))

{(2, 1, 0): 1, (2, 0, 1): 1}

In [357]:
[ w for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[(2, 1, 0), (2, 0, 1)]

In [358]:
[ vector(w) for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[(2, 1, 0), (2, 0, 1)]

In [359]:
[ class_from_weight(R_G, vector(w)) for w, i in parabolic_weight_muliplicities((1, 1)).items() ]

[x0 + x1, 2*x0 - x1]

In [360]:
def chern_classes_on_GP(weight_muliplicities):
    return prod((1 + class_from_weight(R_G, vector(w)))^i for w, i in weight_muliplicities.items())

In [361]:
#ambient spaceを生成
ambient_space_of_G = R_G.space()

#positive rootをambient spaceの元に変換
positive_roots_of_G = [ ambient_space_of_G(pr) for pr in R_G.positive_roots() ]
positive_roots_of_G

[(1, -1, 0), (1, 0, -1), (0, 1, -1)]

In [362]:
# positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
roots = [ pr for pr in R_G.positive_roots() ]
for i in crossed_out_nodes:
    roots = [ pr for pr in roots if not (pr - R_G.simple_roots()[i]).is_positive_root() and pr != R_G.simple_roots()[i] ]

# ambient spaceの元に変換
positive_roots_of_P = [ ambient_space_of_G(pr) for pr in roots ]
positive_roots_of_P

[(0, 1, -1)]

In [363]:
# the weights of the tangenst spaces at torus-fixed points
flag_tangent_weights = set(positive_roots_of_G)-set(positive_roots_of_P)
flag_tangent_weights = [sum( r[l]*x[l] for l in range(len(x))) for r in flag_tangent_weights]
flag_tangent_weights

[x0 - x2, x0 - x1]

In [364]:
dim_flag = len(R_G.positive_roots()) - len(R_L.positive_roots())
dim_flag

2

In [365]:
# homogeneous part
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [366]:
def numerical_integration_by_localization(f):
    random_x = [RealField(1000)(random()) for i in range(G.root_system().ambient_space().dimension())]
    orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(G)]
    top_of_f = homogeneous_part(f, dim_flag)
    denominator_in_localization = prod(flag_tangent_weights)
    return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(L))

In [367]:
import re
def hrr(chern_classes_of_E: list, chern_classes_of_M: list) -> int:
    # The arguments must be the lists {c_1, .. , c_top}
    rank_E = len(chern_classes_of_E)
    dim_M  = len(chern_classes_of_M)
    
    # Define rings where classes lie
    ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..rank_E) ], order=TermOrder('wdeglex', tuple(1..rank_E)))
    ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..dim_M) ], order=TermOrder('wdeglex', tuple(1..dim_M)))

    # Using Singular, calclate universal formula of chern character
    singular.lib('chern.lib')
    r = singular.ring(0, f'(c(1..{rank_E}))', 'dp')
    l = singular.list(f'c(1..{rank_E})')
    ch_str_list = singular.chAll(l, dim_M).sage_structured_str_list()
    chern_character = [ ring_for_ch(rank_E) ] + [ ring_for_ch(re.sub('c\(([0-9]+)\)', 'c\\1_E', s)) for s in ch_str_list ]

    # Using Singular, calculate universal formula of Todd classes
    r = singular.ring(0, f'(c(1..{dim_M}))', 'dp')
    l = singular.list(f'c(1..{dim_M})')
    todd_str_list = singular.todd(l).sage_structured_str_list()
    todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub('c\(([0-9]+)\)', 'c\\1_M', s)) for s in todd_str_list ]
    
    # Return the value of integration
    return numerical_integration_by_localization( 
        sum(
            chern_character[i](chern_classes_of_E)
            * todd_classes[dim_M - i](chern_classes_of_M)
        for i in (0..dim_M))
    )

In [368]:
w = (1, 1)
sum(w[i - 1] * R_G.fundamental_weights()[i] for i in (1..G.rank()))

(2, 1, 0)

In [369]:
cc = chern_classes_on_GP(parabolic_weight_muliplicities(w))

chern_classes_of_E = [
    homogeneous_part(cc, i)
    for i in (1..dim_flag)
    if homogeneous_part(cc, i) != 0
]

chern_classes_of_M = [ 
    homogeneous_part(prod(1+x for x in flag_tangent_weights), i)
    for i in (1..dim_flag) 
]

In [370]:
chern_classes_of_E

[3*x0, 2*x0^2 + x0*x1 - x1^2]

In [371]:
chern_classes_of_M

[2*x0 - x1 - x2, x0^2 - x0*x1 - x0*x2 + x1*x2]

In [372]:
hrr(chern_classes_of_E, chern_classes_of_M)

9

In [373]:
ambient_space_of_G.weyl_dimension(sum(w[i - 1] * R_G.fundamental_weights()[i] for i in (1..G.rank())))

8